# "2020 January One Day Holding Long Short Portfolio BackTest"
> "2020 January Portfolio"
- toc: false
- branch: master
- badges: false
- comments: true
- categories: [long, short, longshort]
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

In [305]:
#hide_input
import numpy as np
import pandas as pd
import seaborn as sns
import os
import time
import gc
import glob
import matplotlib.pyplot as plt

In [306]:
#hide_input
#df_tes=pd.read_csv('D:\\Downloads\\20194dayreturnpred2019train60K.csv')
df_test=pd.read_csv('D:\\Downloads\\sample28K20194dayreturnpred2019train60K.csv')
df_tes=pd.read_csv('D:\\Downloads\\10Q20194dayreturnpred2019train60K.csv')
#df_tes=pd.read_csv('D:\\Downloads\\1day60kvocab20192020testcumulative.csv')

In [307]:
#hide
#df_testp=df_testp.loc[df_testp.label!='nan'].loc[df_testp.Date>=20190101].loc[df_testp.Date<20190201].sort_values(by='Date',ascending=True).copy()
#df_test=df_test.loc[df_test.label!='nan'].loc[df_test.Date>=20190101].loc[df_test.Date<20190201].sort_values(by='Date',ascending=True).copy()
df_tes=df_tes.loc[df_tes.label!='nan'].loc[df_tes.Date>=20200101].loc[df_tes.Date<20200201].sort_values(by='Date',ascending=True).copy().reset_index(drop=True)
df_long=df_tes.loc[df_tes.pred==1].copy().reset_index(drop=True)
df_short=df_tes.loc[df_tes.pred==-1].copy().reset_index(drop=True)
#df_testp.reset_index(drop=True, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [308]:
#hide
#df_testp=df_testp.loc[df_testp.label!='nan'].loc[df_testp.Date>=20190101].loc[df_testp.Date<20190201].sort_values(by='Date',ascending=True).copy()
#df_test=df_test.loc[df_test.label!='nan'].loc[df_test.Date>=20190101].loc[df_test.Date<20190201].sort_values(by='Date',ascending=True).copy()
df_test=df_test.loc[df_test.label!='nan'].loc[df_test.Date>=20200101].loc[df_test.Date<20200201].sort_values(by='Date',ascending=True).copy().reset_index(drop=True)
df_longa=df_test.loc[df_test.pred==1].copy().reset_index(drop=True)
df_shorta=df_test.loc[df_test.pred==-1].copy().reset_index(drop=True)
#df_testp.reset_index(drop=True, inplace=True)

In [309]:
#hide
df_tes['dailyaveragereturn']=0.0
for date in df_tes['Date'].unique():
    df_tes.loc[df_tes['Date']==date,'dailyaveragereturn']=np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))

In [310]:
#hide
df_test['dailyaveragereturn']=0.0
for date in df_test['Date'].unique():
    df_test.loc[df_test['Date']==date,'dailyaveragereturn']=np.prod(1+(df_test.loc[df_test['Date']==date]['labelr1']/100))**(1/len(df_test.loc[df_test['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))

In [311]:
#hide
df_a=df_tes.drop_duplicates(subset = ["Date"],keep='last').reset_index(drop=True)
df_b=df_test.drop_duplicates(subset = ["Date"],keep='last').reset_index(drop=True)

In [312]:
#hide
df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a.loc[l,'cumreturn1b']=np.prod(df_a['dailyaveragereturn'][:l+1])

df_b['cumreturn1c']=0.0
for l in range(len(df_b)):
    df_b.loc[l,'cumreturn1c']=np.prod(df_b['dailyaveragereturn'][:l+1])

In [313]:
#hide
df_a=pd.merge(df_a,df_b[['Date','cumreturn1c']], right_on='Date', left_on='Date',how='inner')

**Long-Short Portfolio**

In [314]:
#hide
if len(df_a)>0:
    import altair as alt
    import datetime
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_a.Date.min()].loc[df_dow.Date<=df_a.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_a=pd.merge(df_a,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_a.Date.min()].loc[df_sp500.Date<=df_a.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_a=pd.merge(df_a,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    #df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    #df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    #df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    #df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [315]:
#hide_input
df_a[['Date','cumreturn1b','cumreturn1c','cumreturndow','cumreturnsp500']]

,Date,cumreturn1b,cumreturn1c,cumreturndow,cumreturnsp500
0,2020-01-02,0.990132,0.987670,0.992940,0.991897
1,2020-01-03,0.992865,0.988496,0.996449,0.994270
2,2020-01-06,0.940697,0.980582,0.993655,0.990124
3,2020-01-07,0.953942,0.977231,0.998527,0.995715
4,2020-01-08,0.943184,0.973115,1.005172,1.003052
5,2020-01-09,0.960060,0.961775,1.002302,0.998440
6,2020-01-13,0.960060,0.980680,1.007766,1.002455
7,2020-01-14,0.975758,0.987822,1.009651,1.005592
8,2020-01-15,0.919344,0.997455,1.018098,1.014855
9,2020-01-16,0.983229,0.999209,1.022030,1.016603


In [316]:
#hide_input
import altair as alt
a1 = alt.Chart(df_a).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1b:Q',
    size='cumreturn1b',
    
).properties(
    width=1000,
    height=1000,
    title='10QPortfolio (blue) vs 8KPortfolio (brown) vs Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_a).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_a).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a4 = alt.Chart(df_a).mark_trail(color='brown').encode(
    x='Date:T',
    y='cumreturn1c:Q',
    size='cumreturn1c',

).properties(
    width=1000,
    height=1000
)

#a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
#    x='Date:T',
#    y='cumreturn1b:Q',

#).properties(
#    width=1000,
#    height=1000
#)

a1 + a2 + a3 + a4# + a5

alt.LayerChart(...)

**Long portfolio**

In [317]:
#hide
df_long['dailyaveragereturn']=0.0
for date in df_long['Date'].unique():
    df_long.loc[df_long['Date']==date,'dailyaveragereturn']=np.prod(1+(df_long.loc[df_long['Date']==date]['labelr1']/100))**(1/len(df_long.loc[df_long['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))

df_longa['dailyaveragereturn']=0.0
for date in df_longa['Date'].unique():
    df_longa.loc[df_longa['Date']==date,'dailyaveragereturn']=np.prod(1+(df_longa.loc[df_longa['Date']==date]['labelr1']/100))**(1/len(df_longa.loc[df_longa['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))

df_a=df_long.drop_duplicates(subset = ["Date"],keep='last').reset_index(drop=True)
df_b=df_longa.drop_duplicates(subset = ["Date"],keep='last').reset_index(drop=True)

df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a.loc[l,'cumreturn1b']=np.prod(df_a['dailyaveragereturn'][:l+1])
    
df_b['cumreturn1c']=0.0
for l in range(len(df_b)):
    df_b.loc[l,'cumreturn1c']=np.prod(df_b['dailyaveragereturn'][:l+1])

df_a=pd.merge(df_a,df_b[['Date','cumreturn1c']], right_on='Date', left_on='Date',how='inner')

In [318]:
#hide
if len(df_a)>0:
    import altair as alt
    import datetime
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_a.Date.min()].loc[df_dow.Date<=df_a.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_a=pd.merge(df_a,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_a.Date.min()].loc[df_sp500.Date<=df_a.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_a=pd.merge(df_a,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    #df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    #df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    #df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    #df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [319]:
#hide_input
df_a[['Date','cumreturn1b','cumreturn1c','cumreturndow','cumreturnsp500']]

,Date,cumreturn1b,cumreturn1c,cumreturndow,cumreturnsp500
0,2020-01-07,1.010270,0.968483,1.004902,1.005647
1,2020-01-29,1.028823,0.902513,1.014358,1.009647
2,2020-01-30,0.993462,0.883546,0.996398,0.988537
3,2020-01-31,1.008243,0.882676,0.996398,0.988537


In [320]:
#hide_input
import altair as alt
a1 = alt.Chart(df_a).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1b:Q',
    size='cumreturn1b',
    
).properties(
    width=1000,
    height=1000,
    title='10QPortfolio (blue) vs 8KPortfolio (brown) vs Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_a).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_a).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a4 = alt.Chart(df_a).mark_trail(color='brown').encode(
    x='Date:T',
    y='cumreturn1c:Q',
    size='cumreturn1c',

).properties(
    width=1000,
    height=1000
)

#a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
#    x='Date:T',
#    y='cumreturn1b:Q',

#).properties(
#    width=1000,
#    height=1000
#)

a1 + a2 + a3 + a4# + a5

alt.LayerChart(...)

**Short Portfolio**

In [321]:
#hide
df_short['dailyaveragereturn']=0.0
for date in df_short['Date'].unique():
    df_short.loc[df_short['Date']==date,'dailyaveragereturn']=np.prod(1+(df_short.loc[df_short['Date']==date]['labelr1']/100))**(1/len(df_short.loc[df_short['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))

df_shorta['dailyaveragereturn']=0.0
for date in df_shorta['Date'].unique():
    df_shorta.loc[df_shorta['Date']==date,'dailyaveragereturn']=np.prod(1+(df_shorta.loc[df_shorta['Date']==date]['labelr1']/100))**(1/len(df_shorta.loc[df_shorta['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))

df_a=df_short.drop_duplicates(subset = ["Date"],keep='last').reset_index(drop=True)
df_b=df_shorta.drop_duplicates(subset = ["Date"],keep='last').reset_index(drop=True)

df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a.loc[l,'cumreturn1b']=np.prod(df_a['dailyaveragereturn'][:l+1])
    
df_b['cumreturn1c']=0.0
for l in range(len(df_b)):
    df_b.loc[l,'cumreturn1c']=np.prod(df_b['dailyaveragereturn'][:l+1])

df_a=pd.merge(df_a,df_b[['Date','cumreturn1c']], right_on='Date', left_on='Date',how='inner')

In [322]:
#hide
if len(df_a)>0:
    import altair as alt
    import datetime
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_a.Date.min()].loc[df_dow.Date<=df_a.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_a=pd.merge(df_a,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_a.Date.min()].loc[df_sp500.Date<=df_a.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_a=pd.merge(df_a,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    #df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    #df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    #df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    #df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [323]:
#hide_input
df_a[['Date','cumreturn1b','cumreturn1c','cumreturndow','cumreturnsp500']]

,Date,cumreturn1b,cumreturn1c,cumreturndow,cumreturnsp500
0,2020-01-02,0.990132,0.980368,0.992940,0.991897
1,2020-01-03,0.992865,0.988184,0.996449,0.994270
2,2020-01-06,0.940697,0.982595,0.993655,0.990124
3,2020-01-07,0.961147,0.978099,0.998527,0.995715
4,2020-01-08,0.950309,0.984451,1.005172,1.003052
5,2020-01-09,0.967312,0.967650,1.002302,0.998440
6,2020-01-13,0.967312,0.988417,1.007766,1.002455
7,2020-01-14,0.983129,0.998036,1.009651,1.005592
8,2020-01-15,0.926288,1.012114,1.018098,1.014855
9,2020-01-16,0.990656,1.013408,1.022030,1.016603


In [324]:
#hide_input
import altair as alt
a1 = alt.Chart(df_a).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1b:Q',
    size='cumreturn1b',
    
).properties(
    width=1000,
    height=1000,
    title='10QPortfolio (blue) vs 8KPortfolio (brown) vs Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_a).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_a).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a4 = alt.Chart(df_a).mark_trail(color='brown').encode(
    x='Date:T',
    y='cumreturn1c:Q',
    size='cumreturn1c',

).properties(
    width=1000,
    height=1000
)

#a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
#    x='Date:T',
#    y='cumreturn1b:Q',

#).properties(
#    width=1000,
#    height=1000
#)

a1 + a2 + a3 + a4# + a5

alt.LayerChart(...)